In [1]:
import torch
import cv2

In [2]:
def get_prediction(img, threshold, model):
    lab = {0:'both', 1:'infection', 2:'ischaemia', 3:'none'}
    pred = model(img) # Pass the image to the model
#     img = Image.open(img_path) # Load the image
#    pred = model(img) # Pass the image to the model
    pred_class = [lab[i] for i in list(pred[0]['labels'].cpu().numpy())] # Get the Prediction Score
    print(pred_class)
    pred_boxes = [[(i[0], i[1]), (i[2], i[3])] for i in list(pred[0]['boxes'].cpu().detach().numpy())] # Bounding boxes
    print(pred_boxes)
    pred_score = list(pred[0]['scores'].cpu().detach().numpy())
    print(pred_score)
    pred_t = [pred_score.index(x) for x in pred_score if x > threshold][-1] # Get list of index with score greater than threshold.
    print(pred_t)
    pred_boxes = pred_boxes[:pred_t+1]
    print(pred_boxes)
    pred_class = pred_class[:pred_t+1]
    print(pred_class)
    return pred_boxes, pred_class

In [3]:
def detect_img(im, threshold, model, rect_th=3, text_size=1, text_th=2):
    boxes, pred_cls = get_prediction(im, threshold, model) # Get predictions
    img = cv2.cvtColor(im.cpu().detach().numpy(), cv2.COLOR_BGR2RGB) # Convert to RGB
    for i in range(len(boxes)):
        img = cv2.rectangle(img, (int(boxes[i][0][0]), int(boxes[i][0][1])), (int(boxes[i][1][0]), int(boxes[i][1][1])),color=(0, 255, 0), thickness=rect_th) # Draw Rectangle with the coordinates
        img = cv2.putText(img,pred_cls[i], (int(boxes[i][0][0]), int(boxes[i][0][1])),  cv2.FONT_HERSHEY_SIMPLEX, text_size, (0,255,0),thickness=text_th) # Write the prediction class
    return img

In [4]:
from PIL import Image
from torchvision import transforms
import torchvision
from torchvision.models.detection.faster_rcnn import FastRCNNPredictor
import matplotlib.pyplot as plt
import matplotlib
matplotlib.use('TkAgg')

data_transform = transforms.Compose([transforms.ToTensor()])
original_img = Image.open(r"C:\object_detection\frcnn_medium_sample_2\391.jpg")
original_img.show()
img = data_transform(original_img)
img = torch.unsqueeze(img, dim=0)

trial = cv2.cvtColor(img.cpu().detach().numpy(), cv2.COLOR_BGR2RGB)
plt.imshow(trial)
plt.show()


# original_img = cv2.imread(r"C:\object_detection\frcnn_medium_sample_2\391.jpg")
# plt.imshow(original_img[:,:,::-1])
# plt.show()
# img_tensor = torch.from_numpy(original_img)
# img_tensor = img_tensor.float()
# img_tensor = torch.unsqueeze(img_tensor, dim=0)
# plt.imshow(img_tensor.detach().numpy()[0,:,:, ::-1])
# plt.show()

PATH = './frcnn_net_evaluate.pth'
device = torch.device("cuda") if torch.cuda.is_available() else torch.device("cpu")
print(device)
model = torchvision.models.detection.fasterrcnn_resnet50_fpn()
in_features = model.roi_heads.box_predictor.cls_score.in_features
model.roi_heads.box_predictor = FastRCNNPredictor(in_features, 4)
model.load_state_dict(torch.load(PATH))
model.to(device)
model.eval()
with torch.no_grad():
    img = img.to(device)
    img = detect_img(img, 0.4, model)
    plt.imshow(img)
    plt.show()


cuda


C:\Users\Karthik\anaconda3\envs\pytorch\lib\site-packages\torch\functional.py:445: UserWarning: torch.meshgrid: in an upcoming release, it will be required to pass the indexing argument. (Triggered internally at  ..\aten\src\ATen\native\TensorShape.cpp:2157.)
  return _VF.meshgrid(tensors, **kwargs)  # type: ignore[attr-defined]
Clipping input data to the valid range for imshow with RGB data ([0..1] for floats or [0..255] for integers).


['infection', 'infection', 'infection', 'infection', 'none', 'none', 'infection', 'ischaemia', 'ischaemia', 'ischaemia', 'none', 'none', 'ischaemia', 'infection', 'none', 'infection', 'ischaemia', 'none', 'infection', 'ischaemia', 'none', 'infection', 'ischaemia', 'ischaemia', 'ischaemia', 'none', 'ischaemia', 'none', 'infection', 'ischaemia', 'none', 'ischaemia', 'ischaemia', 'ischaemia', 'none', 'infection', 'none', 'infection', 'ischaemia', 'ischaemia', 'none', 'ischaemia', 'none', 'infection', 'ischaemia', 'infection', 'ischaemia', 'none', 'ischaemia', 'infection', 'infection', 'none', 'none', 'none', 'none', 'infection', 'infection', 'none', 'infection']
[[(16.793747, 9.349016), (71.07551, 81.5375)], [(151.64581, 76.804794), (213.1229, 165.19559)], [(25.124971, 14.58132), (66.48926, 61.875618)], [(127.3928, 63.558464), (224.0, 195.90312)], [(28.295216, 13.058758), (65.11158, 71.88756)], [(165.31068, 79.72174), (205.9932, 139.59286)], [(163.89629, 75.60928), (207.79489, 134.90901)]